# Loop 7 Analysis: Rotation Optimization and bbox3

The evaluator identified two key techniques NOT yet tried:
1. **Rotation optimization (fix_direction)** - Optimize the rotation angle of the entire configuration
2. **bbox3 optimizer** - A fundamentally different optimization approach

Let's analyze these approaches.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
from shapely.geometry import Polygon
from shapely import affinity

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    vertices = [(tx * cos_a - ty * sin_a + x, tx * sin_a + ty * cos_a + y) for tx, ty in zip(TX, TY)]
    return Polygon(vertices)

print('Functions defined')

Functions defined


In [2]:
# Load the current best baseline
df = pd.read_csv('/home/code/external_data/saspav/santa-2025.csv')
print(f'Loaded {len(df)} rows')
print(df.head())

Loaded 20100 rows
      id                       x                       y  \
0  001_0    s-48.196086194214246     s58.770984615214225   
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   
3  003_0      s1.123655816140301      s0.781101815992563   
4  003_1       s1.23405569584216      s1.275999500663759   

                       deg  
0                    s45.0  
1  s203.629377730656841550  
2   s23.629377730656791812  
3        s111.125132292893  
4         s66.370622269343  


In [3]:
# Implement rotation optimization (fix_direction)
def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bounding box side length at a given rotation angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, s], [-s, c]])
    rotated_points = points @ rot_matrix.T
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation_for_n(df, n):
    """Optimize rotation angle for configuration N."""
    prefix = f'{n:03d}_'
    trees = df[df['id'].str.startswith(prefix)]
    
    if len(trees) != n:
        return None, None, None
    
    # Get all polygon vertices
    all_points = []
    for _, row in trees.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        poly = create_tree_polygon(x, y, deg)
        all_points.extend(list(poly.exterior.coords))
    
    points = np.array(all_points)
    
    # Get convex hull for efficiency
    try:
        hull_points = points[ConvexHull(points).vertices]
    except:
        hull_points = points
    
    # Calculate initial side
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    # Optimize rotation
    result = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0, 90),
        method='bounded'
    )
    
    best_angle = result.x
    best_side = result.fun
    improvement = initial_side - best_side
    
    return initial_side, best_side, best_angle, improvement

print('Rotation optimization function defined')

Rotation optimization function defined


In [4]:
# Test rotation optimization on a few N values
test_ns = [1, 10, 50, 100, 150, 200]
print('Testing rotation optimization on selected N values:\n')

for n in test_ns:
    result = optimize_rotation_for_n(df, n)
    if result[0] is not None:
        initial, best, angle, improvement = result
        print(f'N={n:3d}: Initial={initial:.6f}, Best={best:.6f}, Angle={angle:.4f}°, Improvement={improvement:.9f}')

Testing rotation optimization on selected N values:

N=  1: Initial=0.813173, Best=0.813173, Angle=90.0000°, Improvement=-0.000000051
N= 10: Initial=1.940696, Best=1.940696, Angle=0.0000°, Improvement=-0.000000105
N= 50: Initial=4.247076, Best=4.247077, Angle=90.0000°, Improvement=-0.000000461
N=100: Initial=5.878188, Best=5.878188, Angle=0.0000°, Improvement=-0.000000256
N=150: Initial=7.110529, Best=7.110529, Angle=0.0000°, Improvement=0.000000011
N=200: Initial=8.216619, Best=8.216620, Angle=0.0000°, Improvement=-0.000000440


In [5]:
# Apply rotation optimization to ALL N values and calculate total improvement
print('Applying rotation optimization to all N values...')

total_improvement = 0
improvements = []

for n in range(1, 201):
    result = optimize_rotation_for_n(df, n)
    if result[0] is not None:
        initial, best, angle, improvement = result
        # Score contribution: side^2 / n
        initial_score = initial**2 / n
        best_score = best**2 / n
        score_improvement = initial_score - best_score
        total_improvement += score_improvement
        improvements.append({
            'n': n,
            'initial_side': initial,
            'best_side': best,
            'angle': angle,
            'side_improvement': improvement,
            'score_improvement': score_improvement
        })

print(f'\nTotal score improvement from rotation: {total_improvement:.9f}')
print(f'Current score: 70.659959')
print(f'New score after rotation: {70.659959 - total_improvement:.6f}')

Applying rotation optimization to all N values...



Total score improvement from rotation: -0.000006038
Current score: 70.659959
New score after rotation: 70.659965


In [6]:
# Show top 10 N values with most improvement from rotation
import pandas as pd
imp_df = pd.DataFrame(improvements)
imp_df = imp_df.sort_values('score_improvement', ascending=False)
print('Top 10 N values with most improvement from rotation:')
print(imp_df.head(10).to_string(index=False))

Top 10 N values with most improvement from rotation:
  n  initial_side  best_side     angle  side_improvement  score_improvement
128      6.633064   6.633060  0.000443      3.850915e-06       3.991150e-07
 88      5.555095   5.555094  0.000122      9.701032e-07       1.224776e-07
 87      5.547459   5.547458 89.999934      8.942081e-07       1.140364e-07
178      7.715610   7.715609  0.000097      8.168857e-07       7.081765e-08
176      7.715196   7.715195  0.000034      2.951167e-07       2.587367e-08
132      6.638062   6.638062  0.000015      1.277608e-07       1.284976e-08
130      6.636508   6.636508  0.000011      9.676777e-08       9.880002e-09
177      7.715550   7.715550  0.000007      6.173760e-08       5.382367e-09
 71      5.000850   5.000850  0.000004      2.344306e-08       3.302401e-09
150      7.110529   7.110529  0.000004      1.074813e-08       1.018998e-09


In [7]:
# Summary of rotation optimization potential
print('='*60)
print('ROTATION OPTIMIZATION SUMMARY')
print('='*60)
print(f'Total score improvement: {total_improvement:.9f}')
print(f'N values with improvement > 1e-6: {len(imp_df[imp_df.score_improvement > 1e-6])}')
print(f'N values with improvement > 1e-9: {len(imp_df[imp_df.score_improvement > 1e-9])}')
print(f'Average improvement per N: {total_improvement/200:.12f}')
print('='*60)
print(f'\nConclusion: Rotation optimization provides {total_improvement:.9f} improvement.')
print(f'This is {"significant" if total_improvement > 0.001 else "negligible"} compared to the 1.74 gap to target.')

ROTATION OPTIMIZATION SUMMARY
Total score improvement: -0.000006038
N values with improvement > 1e-6: 0
N values with improvement > 1e-9: 10
Average improvement per N: -0.000000030191

Conclusion: Rotation optimization provides -0.000006038 improvement.
This is negligible compared to the 1.74 gap to target.


In [ ]:
# Let's analyze the bbox3 C++ code to understand its approach
# The key insight is that bbox3 uses fundamentally different techniques:
# 1. Complex number vector coordination
# 2. Fluid dynamics
# 3. Hinge pivot
# 4. Density gradient flow
# 5. Global boundary tension

print("bbox3 code analyzed. Key differences from Eazy optimizer:")
print("1. Uses complex numbers for position representation")
print("2. Simpler SA approach but with multi-scale optimization")
print("3. Parameters: -n (iterations), -r (multiplier)")
print("\nThe bbox3-runner workflow:")
print("1. Run bbox3 with various n,r combinations")
print("2. Apply fix_direction (rotation optimization)")
print("3. Repair overlaps with donor solutions")
print("4. Keep best result")

In [ ]:
# Key insight: The gap to target is 1.74 points (2.5%)
# All local search approaches (SA, Eazy, bbox3) show negligible improvement
# The baseline is at a VERY strong local optimum

# Let's analyze what would be needed to reach the target:
target = 68.919154
current = 70.659959
gap = current - target

print(f"Current score: {current:.6f}")
print(f"Target score: {target:.6f}")
print(f"Gap: {gap:.6f} ({gap/current*100:.2f}%)")
print()

# Per-N analysis: How much improvement needed per N?
avg_improvement_per_n = gap / 200
print(f"Average improvement needed per N: {avg_improvement_per_n:.6f}")
print()

# If we assume improvement is proportional to score contribution:
# N=1 contributes ~0.66 (0.94% of total)
# N=200 contributes ~0.34 (0.48% of total)
# Large N values contribute more to total score

# Let's calculate the theoretical minimum side for each N
# Theoretical minimum = sqrt(n * tree_area) / sqrt(packing_efficiency)
# Tree area ≈ 0.35 (estimated from polygon)
tree_area = 0.35
packing_efficiency = 0.9  # Assume 90% packing efficiency

print("Theoretical analysis:")
for n in [1, 10, 50, 100, 150, 200]:
    theoretical_min = np.sqrt(n * tree_area / packing_efficiency)
    theoretical_score = theoretical_min**2 / n
    print(f"N={n:3d}: Theoretical min side = {theoretical_min:.4f}, score contribution = {theoretical_score:.4f}")